<a href="https://colab.research.google.com/github/ttpss930141011/fast_bert-chinese-classification/blob/master/fast_bert_chinese_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fast-bert
!pip install torch==1.3.0+cu100 torchvision==0.4.2+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install datetime
!pip install fastai
!pip install pytorch-transformers
!pip install python-box
!pip install torch torchvision

     |████████████████████████████████| 92kB 4.4MB/s 
     |████████████████████████████████| 450kB 15.4MB/s 
     |████████████████████████████████| 194kB 63.2MB/s 
     |████████████████████████████████| 860kB 54.0MB/s 
     |████████████████████████████████| 1.0MB 42.3MB/s 
     |████████████████████████████████| 133kB 53.2MB/s 
     |████████████████████████████████| 552kB 62.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=531055738a1ccf09208e5625d3a592f03f4d47ac44e9029761d238bebfdf66de
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 744.5MB 24kB/s 
     |████████████████████████████████| 10.2MB 807kB/s 
ERROR: torchvision 0.4.2+cu100 has requirement torch==1.3.1, but you'll have torch 1.3.0+cu100 which is incompatible.
  Found exist

In [2]:
#@title
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
%cd ..


Cloning into 'apex'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 5613 (delta 9), reused 18 (delta 9), pack-reused 5585
Receiving objects: 100% (5613/5613), 13.36 MiB | 23.84 MiB/s, done.
Resolving deltas: 100% (3632/3632), done.
/content/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-l4x7rxd1
Created temporary directory: /tmp/pip-req-tracker-gqq2ubvv
Created requirements tracker '/tmp/pip-req-tracker-gqq2ubvv'
Created temporary directory: /tmp/pip-install-ybj_z0gr
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-tmsae8rp
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-gqq2ubvv'
    Run

In [3]:
!git clone https://github.com/ttpss930141011/Project.git

Cloning into 'Project'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 310 (delta 15), reused 19 (delta 8), pack-reused 278
Receiving objects: 100% (310/310), 57.11 MiB | 24.29 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [0]:
from box import Box
import torch
import apex
from fastai.text import *
import datetime
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

LOG_PATH=Path('logs/')
if not LOG_PATH.exists():
  LOG_PATH.mkdir()
MODEL_PATH=Path('models/')
if not MODEL_PATH.exists():
  MODEL_PATH.mkdir()
DATA_PATH = Path('Project/sample/data/')     
LABEL_PATH = Path('Project/sample/labels/')   
BERT_PRETRAINED_PATH = "bert-base-chinese"

OUTPUT_PATH=Path('output/')
if not OUTPUT_PATH.exists():
  OUTPUT_PATH.mkdir()
args = Box({
    "run_text": "multilabel toxic comments with freezable layers",
    "train_size": -1,
    "val_size": -1,
    "log_path": LOG_PATH,
    "full_data_dir": DATA_PATH,
    "data_dir": DATA_PATH,
    "task_name": "toxic_classification_lib",
    "no_cuda": False,
    "bert_model": BERT_PRETRAINED_PATH,
    "max_seq_length": 512,
    "output_dir": OUTPUT_PATH,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 8,
    "eval_batch_size": 16,
    "learning_rate": 5e-5,
    "num_train_epochs": 6,
    "warmup_proportion": 0.0,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": True,
    "fp16_opt_level": "O1",
    "weight_decay": 0.0,
    "adam_epsilon": 1e-8,
    "max_grad_norm": 1.0,
    "max_steps": -1,
    "warmup_steps": 500,
    "logging_steps": 50,
    "eval_all_checkpoints": True,
    "overwrite_output_dir": True,
    "overwrite_cache": False,
    "seed": 42,
    "loss_scale": 128,
    "task_name": 'intent'
})


import logging

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

device = torch.device('cuda')

if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

torch.cuda.empty_cache() 


In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [6]:
import torch

torch.version.cuda

'10.0.130'

In [7]:
torch.cuda.is_available()

True

In [8]:
from pytorch_transformers import BertTokenizer
from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc


12/30/2019 19:16:13 - INFO - transformers.file_utils -   PyTorch version 1.3.0+cu100 available.


In [9]:
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, 
                                          do_lower_case=args['do_lower_case'])

12/30/2019 19:16:19 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmp9j8lgft9


100%|██████████| 109540/109540 [00:00<00:00, 808775.02B/s]

12/30/2019 19:16:19 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp9j8lgft9 to cache at /root/.cache/torch/pytorch_transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
12/30/2019 19:16:19 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
12/30/2019 19:16:19 - INFO - pytorch_transformers.file_utils -   removing temp file /tmp/tmp9j8lgft9
12/30/2019 19:16:19 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [0]:
label_cols = ["label1", "label2", "label3" ]

In [11]:
databunch = BertDataBunch(DATA_PATH,
              LABEL_PATH,
              tokenizer = 'bert-base-chinese', 
              train_file = 'train.csv', 
              val_file = 'valid.csv',
              test_data = 'test.csv',
              text_col = "text", 
              label_col = label_cols,
              batch_size_per_gpu = 8, 
              max_seq_length = 512, 
              multi_gpu = multi_gpu, 
              multi_label = True,
              model_type ='bert')

12/30/2019 19:16:19 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpfnfefmu0
12/30/2019 19:16:20 - INFO - transformers.file_utils -   copying /tmp/tmpfnfefmu0 to cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
12/30/2019 19:16:20 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
12/30/2019 19:16:20 - INFO - transformers.file_utils -   removing temp file /tmp/tmpfnfefmu0
12/30/2019 19:16:20 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/t

In [12]:
logger.info(args)

12/30/2019 19:16:23 - INFO - root -   {'run_text': 'multilabel toxic comments with freezable layers', 'train_size': -1, 'val_size': -1, 'log_path': PosixPath('logs'), 'full_data_dir': PosixPath('Project/sample/data'), 'data_dir': PosixPath('Project/sample/data'), 'task_name': 'intent', 'no_cuda': False, 'bert_model': 'bert-base-chinese', 'max_seq_length': 512, 'output_dir': PosixPath('output'), 'do_train': True, 'do_eval': True, 'do_lower_case': True, 'train_batch_size': 8, 'eval_batch_size': 16, 'learning_rate': 5e-05, 'num_train_epochs': 6, 'warmup_proportion': 0.0, 'local_rank': -1, 'seed': 42, 'gradient_accumulation_steps': 1, 'optimize_on_cpu': False, 'fp16': True, 'fp16_opt_level': 'O1', 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'max_steps': -1, 'warmup_steps': 500, 'logging_steps': 50, 'eval_all_checkpoints': True, 'overwrite_output_dir': True, 'overwrite_cache': False, 'loss_scale': 128}


In [0]:
metrics = []

metrics.append({'name': 'accuracy', 'function': accuracy_multilabel})
metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'fbeta', 'function': fbeta})

In [14]:

learner = BertLearner.from_pretrained_model(databunch,
                                            "bert-base-chinese",
                                            metrics=metrics, 
                                            device=device,
                                            logger=logger,
                                            warmup_steps=500,
                                            multi_gpu=multi_gpu,
                                            is_fp16=True, 
                                            multi_label=True,
                                            logging_steps=0,
                                            output_dir=OUTPUT_PATH)

12/30/2019 19:16:23 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json not found in cache or force_download set to True, downloading to /tmp/tmpix45pop_
12/30/2019 19:16:24 - INFO - transformers.file_utils -   copying /tmp/tmpix45pop_ to cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
12/30/2019 19:16:24 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
12/30/2019 19:16:24 - INFO - transformers.file_utils -   removing temp file /tmp/tmpix45pop_
12/30/2019 19:16:24 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cach

In [15]:

learner.fit(4, lr=5e-5,schedule_type="warmup_linear" )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
12/30/2019 19:16:45 - INFO - root -   ***** Running training *****
12/30/2019 19:16:45 - INFO - root -     Num examples = 1849
12/30/2019 19:16:45 - INFO - root -     Num Epochs = 4
12/30/2019 19:16:45 - INFO - root -     Total train batch size (w. parallel, distributed & accumulation) = 8
12/30/2019 19:16:45 - 

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
12/30/2019 19:18:48 - INFO - root -   Running evaluation
12/30/2019 19:18:48 - INFO - root -     Num examples = 391
12/30/2019 19:18:48 - INFO - root -     Batch size = 16


12/30/2019 19:18:55 - INFO - root -   eval_loss after epoch 1: 0.4574952495098114: 
12/30/2019 19:18:55 - INFO - root -   eval_accuracy after epoch 1: 0.8209718670076727: 
12/30/2019 19:18:55 - INFO - root -   eval_accuracy_thresh after epoch 1: 0.937766432762146: 
12/30/2019 19:18:55 - INFO - root -   eval_roc_auc after epoch 1: 0.602981774494517: 
12/30/2019 19:18:55 - INFO - root -   eval_fbeta after epoch 1: 0.1336621642112732: 
12/30/2019 19:18:55 - INFO - root -   lr after epoch 1: 2.32e-05
12/30/2019 19:18:55 - INFO - root -   train_loss after epoch 1: 0.539489910391898
12/30/2019 19:18:55 - INFO - root -   

12/30/2019 19:20:58 - INFO - root -   Running evaluation
12/30/2019 19:20:58 - INFO - root -     Num examples = 391
12/30/2019 19:20:58 - INFO - root -     Batch size = 16


12/30/2019 19:21:06 - INFO - root -   eval_loss after epoch 2: 0.25221720576286316: 
12/30/2019 19:21:06 - INFO - root -   eval_accuracy after epoch 2: 0.22250639386189258: 
12/30/2019 19:21:06 - INFO - root -   eval_accuracy_thresh after epoch 2: 0.9394714832305908: 
12/30/2019 19:21:06 - INFO - root -   eval_roc_auc after epoch 2: 0.7056887605122568: 
12/30/2019 19:21:06 - INFO - root -   eval_fbeta after epoch 2: 0.0: 
12/30/2019 19:21:06 - INFO - root -   lr after epoch 2: 4.64e-05
12/30/2019 19:21:06 - INFO - root -   train_loss after epoch 2: 0.3482200268912932
12/30/2019 19:21:06 - INFO - root -   

12/30/2019 19:23:09 - INFO - root -   Running evaluation
12/30/2019 19:23:09 - INFO - root -     Num examples = 391
12/30/2019 19:23:09 - INFO - root -     Batch size = 16


12/30/2019 19:23:17 - INFO - root -   eval_loss after epoch 3: 0.21121862143278122: 
12/30/2019 19:23:17 - INFO - root -   eval_accuracy after epoch 3: 0.1918158567774936: 
12/30/2019 19:23:17 - INFO - root -   eval_accuracy_thresh after epoch 3: 0.9394714832305908: 
12/30/2019 19:23:17 - INFO - root -   eval_roc_auc after epoch 3: 0.8198601773983283: 
12/30/2019 19:23:17 - INFO - root -   eval_fbeta after epoch 3: 0.0: 
12/30/2019 19:23:17 - INFO - root -   lr after epoch 3: 2.7102803738317755e-05
12/30/2019 19:23:17 - INFO - root -   train_loss after epoch 3: 0.2115914092623982
12/30/2019 19:23:17 - INFO - root -   

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
12/30/2019 19:25:20 - INFO - root -   Running evaluation
12/30/2019 19:25:20 - INFO - root -     Num examples = 391
12/30/2019 19:25:20 - INFO - root -     Batch size = 16


12/30/2019 19:25:28 - INFO - root -   eval_loss after epoch 4: 0.19820553779602051: 
12/30/2019 19:25:28 - INFO - root -   eval_accuracy after epoch 4: 0.4578005115089514: 
12/30/2019 19:25:28 - INFO - root -   eval_accuracy_thresh after epoch 4: 0.9394714832305908: 
12/30/2019 19:25:28 - INFO - root -   eval_roc_auc after epoch 4: 0.861027325477365: 
12/30/2019 19:25:28 - INFO - root -   eval_fbeta after epoch 4: 0.0: 
12/30/2019 19:25:28 - INFO - root -   lr after epoch 4: 0.0
12/30/2019 19:25:28 - INFO - root -   train_loss after epoch 4: 0.18236378647772403
12/30/2019 19:25:28 - INFO - root -   



(928, 0.3204162832558283)

In [16]:
learner.validate()

12/30/2019 19:25:28 - INFO - root -   Running evaluation
12/30/2019 19:25:28 - INFO - root -     Num examples = 391
12/30/2019 19:25:28 - INFO - root -     Batch size = 16


{'accuracy': 0.4578005115089514,
 'accuracy_thresh': 0.9394714832305908,
 'fbeta': 0.0,
 'loss': 0.19820553779602051,
 'roc_auc': 0.861027325477365}

In [17]:
learner.save_model()

12/30/2019 19:25:35 - INFO - transformers.configuration_utils -   Configuration saved in output/model_out/config.json
12/30/2019 19:25:36 - INFO - transformers.modeling_utils -   Model weights saved in output/model_out/pytorch_model.bin


In [18]:

texts1 = [
  "我想吃東西",
  
] 
texts2 = [

  "展現 真正 的 自己 很 重要 "
]

predictions1 = learner.predict_batch(texts1)
predictions2 = learner.predict_batch(texts2)

12/30/2019 19:25:36 - INFO - root -   Writing example 0 of 1
12/30/2019 19:25:36 - INFO - root -   Writing example 0 of 1


In [19]:
predictions1


[[('label1', 0.1181640625),
  ('label2', 0.1033935546875),
  ('label3', 0.04486083984375)]]

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [21]:
predictions2

[[('label2', 0.10430908203125),
  ('label1', 0.0970458984375),
  ('label3', 0.0469970703125)]]